# BatchIterator

In [20]:
# !pip install git+https://github.com/DIAGNijmegen/pathology-whole-slide-data@main

In [21]:
import os
if os.name =='nt':
    os.add_dll_directory(r"C:\Program Files\openslide\bin") # windows
from wholeslidedata.iterators import create_batch_iterator
from wholeslidedata.visualization.plotting import plot_batch
from dicfg.magics import ConfigMagics
from wholeslidedata.samplers.callbacks import BatchCallback
from wholeslidedata.samplers.utils import crop_data

from batchgenerators.transforms.abstract_transforms import AbstractTransform, Compose
from batchgenerators.transforms.color_transforms import BrightnessMultiplicativeTransform, \
    ContrastAugmentationTransform, GammaTransform
from batchgenerators.transforms.noise_transforms import GaussianNoiseTransform, GaussianBlurTransform
from batchgenerators.transforms.resample_transforms import SimulateLowResolutionTransform
from batchgenerators.transforms.spatial_transforms import SpatialTransform, MirrorTransform
from batchgenerators.transforms.utility_transforms import RemoveLabelTransform, RenameTransform, NumpyToTensor

# This will allow loading yaml in an notebook cell
ConfigMagics.register_magics()

import matplotlib.pyplot as plt

import numpy as np

from multiprocessing import freeze_support

import time

In [22]:
# pip3 install -e /data/pathology/projects/pathology-lung-TIL/nnUNet_v2
from nnunetv2.training.data_augmentation.custom_transforms.pathology_transforms import HedTransform, HsvTransform, Clip01

In [23]:
from wholeslidedata.iterators.batchiterator import BatchIterator

# class WholeSlidePlainnnUnetBatchIterator(BatchIterator):
#     def __next__(self):
#         x_batch, y_batch, _ = super().__next__()
#         x_batch = x_batch.transpose(0,3,1,2).astype('float32')#/255
#         y_batch = y_batch.astype('int16')
#         return {'data': x_batch, 'seg': y_batch}

In [63]:
import torch

In [86]:
class WholeSlidePlainnnUnetBatchIterator(BatchIterator):
    def __next__(self):
        x_batch, y_batch, *extras, _ = super().__next__()
        x_batch = crop_data(x_batch, [512, 512])
        y_batch = crop_data(np.expand_dims(y_batch, -1), [512, 512]).squeeze()
        print(self.patch_size)

        data = torch.FloatTensor(x_batch.transpose(0,3,1,2) /255.).to(device)
        target = [torch.FloatTensor(np.expand_dims(y_batch, 1)).to(device)] + [
            torch.FloatTensor(np.expand_dims(extra, 1)).to(device) 
            for extra in extras]         
        return {'data': data, 'target': target}

In [127]:
class Check(AbstractTransform):
    def __init__(self, data_key="data", label_key="seg", property_key="properties", p_per_sample=1, string=''):
        self.data_key = data_key
        self.label_key = label_key
        self.property_key = property_key
        self.string = string
    def __call__(self, **data_dict):
        data = data_dict.get(self.data_key)
        print(np.min(data), np.max(data))
        print(data.dtype)
        # seg = data_dict.get(self.label_key)
        # props = data_dict.get(self.property_key)
        if self.string:
            print(self.string)
        # print('\n\n\nPROPERTIES BEFORE TRANSFORMS', props, '\n\n\n')
        return data_dict
###

In [128]:
class nnUnetBatchCallback(BatchCallback):
    
    # patch_size_spatial (width/height)
    def __init__(self, patch_size_spatial, check=False):
        tr_transforms = []
        rotation_for_DA= {'x': (-3.141592653589793, 3.141592653589793), 'y': (0, 0), 'z': (0, 0)}
        
        if check:
            tr_transforms.append(Check())
        
        tr_transforms.append(SpatialTransform(
            patch_size_spatial, 
            patch_center_dist_from_border=None,
            do_elastic_deform=False,
            alpha=(0, 0),
            sigma=(0, 0),
            do_rotation=True,
            angle_x=rotation_for_DA['x'],
            angle_y=rotation_for_DA['y'],
            angle_z=rotation_for_DA['z'],
            p_rot_per_axis=1,  # todo experiment with this
            do_scale=True,
            scale=(0.7, 1.4),
            border_mode_data="constant",
            border_cval_data=0,
            order_data=3,
            border_mode_seg="constant",
            border_cval_seg=-1,
            order_seg=1,
            random_crop=False,  # random cropping is part of our dataloaders
            p_el_per_sample=0,
            p_scale_per_sample=0.2,
            p_rot_per_sample=0.2,
            independent_scale_for_each_axis=False  # todo experiment with this
        ))

        if check:
            tr_transforms.append(Check(string='^ spatial'))
        
        ####
        if True: #do_hed:
            tr_transforms.append(HedTransform(factor=0.05))
        # if True: #do_hsv:
        #     tr_transforms.append(HsvTransform(h_lim=0.10, s_lim=0.10, v_lim=0.10))
        ####
        
        tr_transforms.append(GaussianNoiseTransform(p_per_sample=0.1))
        
        if check:
            tr_transforms.append(Check())
        
        tr_transforms.append(GaussianBlurTransform((0.5, 1.), different_sigma_per_channel=True, p_per_sample=0.2,
                                                   p_per_channel=0.5))
        
        if check:
            tr_transforms.append(Check(string='^ gauss noise and blur'))
        
        tr_transforms.append(BrightnessMultiplicativeTransform(multiplier_range=(0.75, 1.25), p_per_sample=0.15))
        
        if check:
            tr_transforms.append(Check(string='^ bright'))
        
        
        
        tr_transforms.append(ContrastAugmentationTransform(p_per_sample=0.15))
        
        if check:
            tr_transforms.append(Check(string='^ cont'))
            
        tr_transforms.append(SimulateLowResolutionTransform(zoom_range=(0.5, 1), per_channel=True,
                                                            p_per_channel=0.5,
                                                            order_downsample=0, order_upsample=3, p_per_sample=0.25,
                                                            ignore_axes=None))
        
        if check:
            tr_transforms.append(Check(string='^ res'))
            
        tr_transforms.append(GammaTransform((0.7, 1.5), True, True, retain_stats=True, p_per_sample=0.1))
        
        if check:
            tr_transforms.append(Check())
            
        tr_transforms.append(GammaTransform((0.7, 1.5), False, True, retain_stats=True, p_per_sample=0.3))
        
        if check:
            tr_transforms.append(Check(string='^ gamm'))
        
        tr_transforms.append(Clip01())
        
        tr_transforms.append(MirrorTransform((0,1)))
        
        if check:
            tr_transforms.append(Check())
            
        tr_transforms.append(RemoveLabelTransform(-1, 0))
        
        if check:
            tr_transforms.append(Check(string='^ mir, rem lab'))
            
        tr_transforms.append(RenameTransform('seg', 'target', True))
        
        
        self._transforms = Compose(tr_transforms)
    
    def __call__(self, x_batch, y_batch):
        # format to nnUNet
        x_batch = np.stack([x/255  for x in x_batch]).transpose((0, 3, 1, 2)).astype('float32')
        y_batch = np.expand_dims(np.stack(y_batch).astype('int8'), 1)
        
        # transform
        batch = self._transforms(**{'data': x_batch, 'seg': y_batch})
        
        # format back to wsd
        x_batch, y_batch = batch['data'], batch['target']
        x_batch = np.multiply(x_batch, 255).astype(np.uint8)
        return x_batch.transpose((0, 2, 3, 1)), y_batch.squeeze()

In [131]:
#     def get_validation_transforms(deep_supervision_scales: Union[List, Tuple],
#                                   is_cascaded: bool = False,
#                                   foreground_labels: Union[Tuple[int, ...], List[int]] = None,
#                                   regions: List[Union[List[int], Tuple[int, ...], int]] = None,
#                                   ignore_label: int = None) -> AbstractTransform:
#         val_transforms = []
#         val_transforms.append(RemoveLabelTransform(-1, 0))

#         if is_cascaded:
#             val_transforms.append(MoveSegAsOneHotToData(1, foreground_labels, 'seg', 'data'))

#         val_transforms.append(RenameTransform('seg', 'target', True))

#         if regions is not None:
#             # the ignore label must also be converted
#             val_transforms.append(ConvertSegmentationToRegionsTransform(list(regions) + [ignore_label]
#                                                                         if ignore_label is not None else regions,
#                                                                         'target', 'target'))

#         if deep_supervision_scales is not None:
#             val_transforms.append(DownsampleSegForDSTransform2(deep_supervision_scales, 0, input_key='target',
#                                                                output_key='target'))

#         val_transforms.append(NumpyToTensor(['data', 'target'], 'float'))
#         val_transforms = Compose(val_transforms)
#         return val_transforms

In [132]:
# 

:::{seealso}
 - For more information about the **mode**, please see the user guide about [modes](./modes)
 - For an explanation about the **user_config** please see the [batchiterator configuration](/userguide/index.html#batchiterator-configuration)
:::

In [133]:
patch_size

[512, 512]

In [134]:
%matplotlib inline

In [135]:
# %%yaml user_config 

# wholeslidedata:
#     default:
#         yaml_source:
#             training:
#             -   wsi: 
#                     path: C:\Users\joeyspronck\Documents\Other\wsirois\wsi-level-annotations\images\100B.tif
#                 wsa: 
#                     path: C:\Users\joeyspronck\Documents\Other\wsirois\wsi-level-annotations\annotations-tissue-cells-xmls\100B.xml
                        
#         labels:
#             invasive tumor: 1
#             tumor-associated stroma: 2
#             in-situ tumor: 3
#             healthy glands: 4
#             necrosis not in-situ: 5
#             inflamed stroma: 6
#             rest: 7
            
#         batch_shape:
#             batch_size: 2
#             spacing: 1.0
#             shape: [512, 512, 3]
            
#         batch_callbacks: 
#             - "*object": __main__.NNUnetBatchCallback
#               patch_size_spatial: [512,512]

In [136]:
patch_size = [512, 512]
batch_size = 2
sample_double = True

ds_scales = [[1.0, 1.0], [0.5, 0.5], [0.25, 0.25], [0.125, 0.125], [0.0625, 0.0625], [0.03125, 0.03125], [0.015625, 0.015625]]
ds_shapes = [list(np.round([int(i * j) for i, j in zip(patch_size, k)])) for k in ds_scales]

if sample_double:
    ds_sizes = [[shape*2 for shape in ds_shape] for ds_shape in ds_shapes]
    extra_ds_sizes = ds_sizes[1:]
    ds_shapes = tuple([tuple([batch_size]+[shape*2 for shape in ds_shape]) for ds_shape in ds_shapes])
    extra_ds_shapes = ds_shapes[1:]
else:
    ds_sizes = [ds_shape for ds_shape in ds_shapes]
    extra_ds_sizes = ds_sizes[1:]
    ds_shapes = tuple([tuple([batch_size]+ds_shape) for ds_shape in ds_shapes])
    extra_ds_shapes = ds_shapes[1:]


In [137]:
ds_sizes, ds_shapes

([[1024, 1024],
  [512, 512],
  [256, 256],
  [128, 128],
  [64, 64],
  [32, 32],
  [16, 16]],
 ((2, 1024, 1024),
  (2, 512, 512),
  (2, 256, 256),
  (2, 128, 128),
  (2, 64, 64),
  (2, 32, 32),
  (2, 16, 16)))

In [138]:
extra_ds_sizes

[[512, 512], [256, 256], [128, 128], [64, 64], [32, 32], [16, 16]]

In [177]:
%%yaml user_config 

wholeslidedata:
    default:
        backend: 
            'asap'
        
        yaml_source:
            training:
            -   wsi: 
                    path: /data/pathology/projects/tiger/new_structure/training/wsirois/wsi-level-annotations/images/100B.tif
                wsa: 
                    path: /data/pathology/projects/tiger/new_structure/training/wsirois/wsi-level-annotations/annotations-tissue-cells-xmls/100B.xml
                        
        labels:
            invasive tumor: 1
            tumor-associated stroma: 2
            in-situ tumor: 3
            healthy glands: 4
            necrosis not in-situ: 5
            inflamed stroma: 6
            rest: 7
            
        batch_shape:
            batch_size: 2
            spacing: 1.0
            shape: [1024, 1024, 3]
            
        batch_callbacks: 
            # - "*object": __main__.NormalizeSampleCallback
            - "*object": __main__.nnUnetBatchCallback
              patch_size_spatial: [1024,1024]
            - "*object": wholeslidedata.samplers.callbacks.DeepSupervisionBatchCallback
              sizes: [[512, 512], [256, 256], [128, 128], [64, 64], [32, 32], [16, 16]]
                


In [178]:
user_config

{'wholeslidedata': {'default': {'backend': 'asap',
   'yaml_source': {'training': [{'wsi': {'path': '/data/pathology/projects/tiger/new_structure/training/wsirois/wsi-level-annotations/images/100B.tif'},
      'wsa': {'path': '/data/pathology/projects/tiger/new_structure/training/wsirois/wsi-level-annotations/annotations-tissue-cells-xmls/100B.xml'}}]},
   'labels': {'invasive tumor': 1,
    'tumor-associated stroma': 2,
    'in-situ tumor': 3,
    'healthy glands': 4,
    'necrosis not in-situ': 5,
    'inflamed stroma': 6,
    'rest': 7},
   'batch_shape': {'batch_size': 2, 'spacing': 1.0, 'shape': [1024, 1024, 3]},
   'batch_callbacks': [{'*object': '__main__.nnUnetBatchCallback',
     'patch_size_spatial': [1024, 1024]},
    {'*object': 'wholeslidedata.samplers.callbacks.DeepSupervisionBatchCallback',
     'sizes': [[512, 512],
      [256, 256],
      [128, 128],
      [64, 64],
      [32, 32],
      [16, 16]]}]}}}

In [89]:
# from wholeslidedata.iterators.batchiterator import BatchIterator

# class WholeSlidePlainnnUnetBatchIterator(BatchIterator):
#     def __next__(self):
#         x_batch, y_batch, *extras, _ = super().__next__()
#         data = x_batch.transpose(0,3,1,2).astype('float32')
#         target = [np.expand_dims(y_batch, 1).astype('int16')] + [np.expand_dims(extra, 1).astype('int16') for extra in extras]         
#         return {'data': data, 'target': target}

In [90]:
# #need to run the config cell again to correctly set new seed
# seed = int(str(time.time())[-3:])
# print(seed)
# user_config['wholeslidedata']['default']['seed']=seed
# with create_batch_iterator(user_config=user_config, 
#                            context='spawn' if os.name=='nt' else 'fork',
#                            mode='training',
#                            buffer_dtype=np.uint8,
#                            extras_shapes=extra_ds_shapes
#                            # iterator_class=WholeSlidePlainnnUnetBatchIterator
#                           ) as training_batch_generator:
#     for i in range(5):
#         x_batch, y_batch, *extras, _  =  next(training_batch_generator)
#         # break
#         # break
#         batch = x_batch, y_batch
#         print(i)
#         plot_batch(batch[0], batch[1])
#         cropped_batch = [crop_data(batch[0], [512, 512]), crop_data(np.expand_dims(batch[1], -1), [512, 512]).squeeze()]
#         plot_batch(cropped_batch[0], cropped_batch[1])
#         for extra in extras:
#             cropped_extra = crop_data(np.expand_dims(extra, -1), [shape//2 for shape in extra.shape[-2:]]).squeeze()
#             print(extra.shape, cropped_extra.shape)

In [121]:
def half_crop(data):
    cropx = (data.shape[1] - data.shape[1]//2) // 2
    cropy = (data.shape[2] - data.shape[2]//2) // 2
    if len(data.shape) == 3:
        return data[:, cropx:-cropx, cropy:-cropy]
    if len(data.shape) == 4:
        return data[:, cropx:-cropx, cropy:-cropy, :]

In [101]:
crop_data(x_batch, [512, 512]).shape

-255 256


(2, 512, 512, 3)

In [188]:
if torch.cuda.is_available():
    # Get the default CUDA device
    device = torch.cuda.current_device()

    # Get the GPU properties
    gpu_properties = torch.cuda.get_device_properties(device)

    # Retrieve the GPU memory size in bytes
    gpu_memory_bytes = gpu_properties.total_memory

    # Convert bytes to gigabytes
    gpu_memory_gb = gpu_memory_bytes / 1e9

    print(f"GPU Memory Size: {gpu_memory_gb:.2f} GB")
else:
    print("No GPU available.")

GPU Memory Size: 11.55 GB


In [196]:
int(11.9)

11

In [189]:
import multiprocessing

cpu_count = multiprocessing.cpu_count()
print(f"Number of CPUs: {cpu_count}")

Number of CPUs: 88


In [194]:
def get_cpu_limit():
    with open("/sys/fs/cgroup/cpu/cpu.cfs_quota_us") as fp:
        cfs_quota_us = int(fp.read())
    with open("/sys/fs/cgroup/cpu/cpu.cfs_period_us") as fp:
        cfs_period_us = int(fp.read())
    container_cpus = cfs_quota_us // cfs_period_us
    # For physical machine, the `cfs_quota_us` could be '-1'
    cpus = cpu_count() if container_cpus < 1 else container_cpus
    return cpus

In [195]:
get_cpu_limit()

8

In [96]:
data = x_batch
data.shape

(2, 1024, 1024, 3)

In [190]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the default CUDA device
    device = torch.cuda.current_device()

    # Get the GPU properties
    gpu_properties = torch.cuda.get_device_properties(device)

    # Retrieve the GPU memory size in bytes
    gpu_memory_bytes = gpu_properties.total_memory

    # Convert bytes to gigabytes
    gpu_memory_gb = gpu_memory_bytes / 1e9

    print(f"GPU Memory Size: {gpu_memory_gb:.2f} GB")
else:
    print("No GPU available.")

GPU Memory Size: 11.55 GB


In [192]:
import psutil

def get_cpu_limit():
    # Get the number of available logical CPUs
    cpus = psutil.cpu_count(logical=True)

    # Get the CPU affinity of the current process
    cpu_affinity = psutil.Process().cpu_affinity()

    # If CPU affinity is set, return the number of CPUs in the affinity set
    if cpu_affinity:
        return len(cpu_affinity)

    # Return the total number of available CPUs if no CPU affinity is set
    return cpus

get_cpu_limit()

88

In [ ]:
cropx = (data.shape[0] - 512) // 2
cropy = (data.shape[1] - output_shape[1]) // 2

In [91]:
device = torch.device('cuda')

In [112]:
#need to run the config cell again to correctly set new seed
seed = int(str(time.time())[-3:])
print(seed)
user_config['wholeslidedata']['default']['seed']=seed
with create_batch_iterator(user_config=user_config, 
                           context='spawn' if os.name=='nt' else 'fork',
                           mode='training',
                           buffer_dtype=np.uint8,
                           extras_shapes=extra_ds_shapes,
                           iterator_class=WholeSlidePlainnnUnetBatchIterator
                          ) as training_batch_generator:
    for i in range(1):
        batch  =  next(training_batch_generator)
        

814


AttributeError: 'int' object has no attribute 'cast'

In [113]:
x_batch, y_batch, *extras, _  =  next(training_batch_generator)
# break
# break
batch = x_batch, y_batch
print(i)
plot_batch(batch[0], batch[1])
cropped_batch = [crop_data(batch[0], [512, 512]), crop_data(np.expand_dims(batch[1], -1), [512, 512]).squeeze()]
plot_batch(cropped_batch[0], cropped_batch[1])
for extra in extras:
    cropped_extra = crop_data(np.expand_dims(extra, -1), [shape//2 for shape in extra.shape[-2:]]).squeeze()
    print(extra.shape, cropped_extra.shape)

OSError: handle is closed

In [114]:
x_batch.shape

(2, 1024, 1024, 3)

In [115]:
crop_data(x_batch, [512, 512]).shape

-255 256


(2, 512, 512, 3)

In [116]:
half_crop(x_batch).shape

(2, 512, 512, 3)

In [122]:
y_batch.shape

(2, 1024, 1024)

In [123]:
crop_data(np.expand_dims(y_batch, -1), [512, 512]).squeeze().shape

-255 256


(2, 512, 512)

In [124]:
half_crop(y_batch).shape

(2, 512, 512)

In [125]:
[extra.shape for extra in extras]

[(2, 512, 512),
 (2, 256, 256),
 (2, 128, 128),
 (2, 64, 64),
 (2, 32, 32),
 (2, 16, 16)]

In [60]:
extra_ds_sizes

[[512, 512], [256, 256], [128, 128], [64, 64], [32, 32], [16, 16]]

In [59]:
[crop_data(np.expand_dims(extra, -1), [512, 512]).squeeze().shape for extra in extras]

[(2, 512, 512),
 (2, 256, 256),
 (2, 128, 128),
 (2, 64, 64),
 (2, 32, 32),
 (2, 16, 16)]

In [37]:
crop_data(x_batch.shape

(2, 1024, 1024, 3)

In [35]:
x_batch, y_batch, *extras, _  

(array([[[[230, 240, 254],
          [230, 240, 254],
          [229, 240, 253],
          ...,
          [227, 240, 252],
          [226, 240, 251],
          [226, 240, 251]],
 
         [[230, 240, 254],
          [230, 240, 254],
          [229, 240, 253],
          ...,
          [229, 240, 254],
          [228, 240, 253],
          [227, 240, 252]],
 
         [[229, 240, 253],
          [229, 240, 253],
          [229, 240, 253],
          ...,
          [229, 240, 254],
          [228, 240, 253],
          [227, 240, 252]],
 
         ...,
 
         [[127,  97, 142],
          [154, 122, 171],
          [181, 149, 201],
          ...,
          [173, 152, 201],
          [175, 155, 202],
          [179, 164, 207]],
 
         [[151, 119, 171],
          [178, 145, 200],
          [196, 164, 220],
          ...,
          [171, 149, 194],
          [170, 151, 193],
          [180, 164, 206]],
 
         [[174, 144, 200],
          [199, 167, 226],
          [208, 174, 232],
   

In [69]:
extra.shape[-2:]

(16, 16)

In [58]:
y_batch.shape

(2, 1024, 1024)

In [29]:
extra.shape

(2, 16, 16)

In [31]:
np.expand_dims(extra, -1).shape

(2, 16, 16, 1)

In [38]:
crop_data(np.expand_dims(extra, -1), [8, 8]).squeeze()

array([[[0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 2],
        [0, 0, 0, 0, 2, 1, 2, 2],
        [0, 0, 0, 0, 1, 2, 2, 2],
        [0, 0, 0, 0, 2, 2, 2, 2],
        [0, 0, 0, 0, 2, 2, 2, 2]],

       [[0, 1, 2, 2, 2, 2, 1, 1],
        [0, 1, 2, 2, 2, 1, 1, 1],
        [0, 1, 1, 2, 2, 2, 2, 2],
        [0, 1, 1, 1, 2, 2, 2, 1],
        [0, 1, 1, 1, 1, 2, 2, 2],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]], dtype=uint8)

In [39]:
extra.shape

(2, 16, 16)

In [32]:
cropped_extra

array([], shape=(2, 0, 0), dtype=uint8)

In [45]:
[shape//2 for shape in extra.shape[-2:]]

[256, 256]

In [46]:
for extra in extras:
    cropped_extra = crop_data(np.expand_dims(extra, -1), [shape//2 for shape in extra.shape[-2:]]).squeeze()
    print(extra.shape, cropped_extra.shape)

(2, 512, 512) (2, 256, 256)
(2, 256, 256) (2, 128, 128)
(2, 128, 128) (2, 64, 64)
(2, 64, 64) (2, 32, 32)
(2, 32, 32) (2, 16, 16)
(2, 16, 16) (2, 8, 8)


In [14]:
batch[0][0].dtype

dtype('uint16')

In [ ]:
crop_data(batch[0], [512, 512]).shape

In [11]:
len(batch)

3

In [12]:
plt.imshow(batch['data'][0].transpose(1, 2, 0))

TypeError: list indices must be integers or slices, not str

In [ ]:
batch

In [ ]:
batch['data'][0]

In [ ]:
x_ex = [np.ones((512, 512, 3)), np.ones((512, 512, 3))]

In [ ]:
np.stack([x for x in x_ex]).transpose((0, 3, 1, 2)).shape

In [ ]:
np.concatenate(x_ex, axis=0).shape

In [ ]:
batch['data'].shape

In [ ]:
batch['target'].shape

In [ ]:
batch

In [ ]:
batch['data'].shape

In [ ]:
batch['seg'].shape

In [ ]:
x_batch.shape

In [ ]:
y_batch.shape